In [1]:
import subprocess 
from subprocess import PIPE
import rasterio
import json
import glob 
import pandas as pd
import os
import numpy as np
import cv2

In [13]:
imagelist = []
rootdir = "C:/Users/User/Desktop/project/Flood Detection using Satelite Images & Deep Machine Learning Project/testing/image" # define the path for the folder
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        imagelist.append(d)
        
        
print("The number of testing samples is = {len(imagelist)}")

The number of testing samples is = {len(imagelist)}


In [14]:
def image_label(product_id):
    
    rootdir = "sen12floods_s2_labels"
    pd = product_id.split("_")
    pd = "_" + pd[3] + "_" + pd[4] + "_" + pd[5] + "_" + pd[6]
    
    json_data=open("sen12floods_s2_labels/" + rootdir + pd +"/stac.json", "rb")
    jdata = json.load(json_data)
    flood = jdata["properties"]["FLOODING"]
    

    if (flood == "False"):
        image_label = 0
    else:
        image_label = 1
    #print(image_label)
    return image_label

In [15]:
def product_name(path):
    
    json_data=open(path+"/stac.json", "rb")
    jdata = json.load(json_data)
    
    return  jdata['id']

In [17]:
def load_data():
    
    data = [] 
    images = []
    labels = []
    
    for folder in imagelist:
        product_id = product_name(folder)
        label = image_label(product_id)
    
        # Open the img
        image = cv2.imread(folder + "/stack.tif")
        # Append the image and its corresponding label to the output
        images.append(image)
        labels.append(label)

        
        
    images = np.array(images, dtype = 'float32')
    labels = np.array(labels, dtype = 'int32')
        
    #print(images, labels)    
    data.append([images, labels])     

    return images, labels

In [20]:
import keras
model = keras.models.load_model("C:/Users/User/Desktop/project/Flood Detection using Satelite Images & Deep Machine Learning Project/my_model_raw")

In [21]:
test_images, test_labels = load_data()

In [22]:
test_loss = model.evaluate(test_images, test_labels)

1/1 [==============================] - 1s 505ms/step - loss: 0.6508 - accuracy: 0.5000
